# FieldMesh Examples

In [ ]:
from pmd_beamphysics import FieldMesh


import numpy as np

# Nicer plotting
import matplotlib.pyplot as plt

In [ ]:
FM = FieldMesh("../data/solenoid.h5")
FM.geometry

Built-in plotting:

In [ ]:
FM.plot("B", aspect="equal")

On-axis field plotting

In [ ]:
FM.plot_onaxis()

Off-axis plotting can be done with the `axis_values` method to get interpolated values, and manually plot:

In [ ]:
z, Bz = FM.axis_values("z", "abs_B", r=0.04)
plt.plot(z, Bz)
plt.xlabel(r"$z$ (m)")
plt.ylabel(r"$\left|B\right|$ (T)")
plt.show()

# Interpolation

Arbirtrary points can also be interpolated. Here is the value of $B_z$ at `r=0.01`, `theta=0`, `z=0`:

In [ ]:
FM.interpolate("Bz", (0.01, 0, 0))

In [ ]:
FM.interpolate("Bz", [(0, 0, z) for z in np.linspace(-0.1, 0.1, 3)])

Note that the points are orderd by the axis labels:

In [ ]:
FM.axis_labels

# Internal data

attributes and components

In [ ]:
FM.attrs, FM.components.keys()

# Properties

Convenient access to these

In [ ]:
FM.shape

In [ ]:
FM.frequency

Coordinate vectors: `.r`, `.theta`, `.z`, etc.

In [ ]:
FM.r, FM.dr

Grid info

In [ ]:
FM.mins, FM.maxs, FM.deltas

Convenient logicals

In [ ]:
FM.is_static, FM.is_pure_magnetic, FM.is_pure_magnetic, FM.is_pure_electric

# Components

In [ ]:
FM.components

Convenient access to component data

In [ ]:
FM.Bz is FM["magneticField/z"]

Setting .scale will set the underlying attribute

In [ ]:
FM.scale = 2
FM.attrs["fieldScale"], FM.scale

Raw components accessed by their full key

In [ ]:
FM["magneticField/z"]

Scaled component accessed by shorter keys, e.g. 

In [ ]:
FM["Bz"]

In [ ]:
FM["magneticField/z"].max(), FM["Bz"].max()

# Oscillating fields

Oscillating fields have `.harmonic > 0`

In [ ]:
FM = FieldMesh("../data/rfgun.h5")
FM.plot("re_E", aspect="equal", figsize=(12, 4))

The magnetic field is out of phase, so use the `im_` syntax:

In [ ]:
FM.plot("im_Btheta", aspect="equal", figsize=(12, 4))

Max on-axis field:

In [ ]:
np.abs(FM.Ez[0, 0, :]).max()

## Verify the oscillation

Complex fields oscillate as $e^{-i\omega t}$. For TM fields, the spatial components $E_z$ and $B_\theta$ near the axis 

$\Re E_{z} = -\frac{r}{2}\frac{\omega}{c^2} \Im B_\theta$


In [ ]:
c_light = 299792458.0

dr = FM.dr
omega = FM.frequency * 2 * np.pi

# Check the first off-axis grid points
z0 = FM.z
Ez0 = np.real(FM.Ez[0, 0, :])
B1 = -np.imag(FM.Btheta[1, 0, :])

plt.plot(z0, Ez0, label=r"$\Re \left( E_z\right)$")
plt.plot(
    z0,
    B1 * 2 / dr * c_light**2 / omega,
    "--",
    label=r"$-\frac{r}{2}\frac{\omega}{c^2} \Im\left(B_\theta\right)$",
)
plt.ylabel("field (V/m)")
plt.xlabel("z (m)")
plt.legend()
plt.title(r"Complex field oscillation")

# Units

In [ ]:
FM.units("Bz")

This also works:

In [ ]:
FM.units("abs_Ez")

# Write

In [ ]:
FM.write("rfgun2.h5")

Read back and make sure the data are the same.

In [ ]:
FM2 = FieldMesh("rfgun2.h5")

assert FM == FM2

Write to open HDF5 file and test reload:

In [ ]:
import h5py

with h5py.File("test.h5", "w") as h5:
    FM.write(h5, name="myfield")
    FM2 = FieldMesh(h5=h5["myfield"])
    assert FM == FM2

## Write Astra 1D

Astra primarily uses simple 1D (on-axis) fieldmaps.

In [ ]:
FM.write_astra_1d("astra_1d.dat")

Another method returns the array data with some annotation

In [ ]:
FM.to_astra_1d()

## Write Impact-T 

Impact-T uses a particular Fourier representation for 1D fields. These routines form this data.

In [ ]:
idata = FM.to_impact_solrf()
idata.keys()

This is an element that can be used with LUME-Impact

In [ ]:
idata["ele"]

This is a line that would be used 

In [ ]:
idata["line"]

Data that would be written to the rfdata999 file

In [ ]:
idata["rfdata"]

This is the fieldmap that makes that data:

In [ ]:
fmap = idata["fmap"]
fmap.keys()

Additional info:

In [ ]:
fmap["info"]

In [ ]:
from pmd_beamphysics.interfaces.impact import fourier_field_reconsruction

L = np.ptp(z0)
zlist = np.linspace(0, L, len(Ez0))
fcoefs = fmap["field"]["Ez"]["fourier_coefficients"]
reconstructed_Ez0 = np.array(
    [fourier_field_reconsruction(z, fcoefs, z0=-L, zlen=2 * L) for z in zlist]
)

fig, ax = plt.subplots()
ax2 = ax.twinx()
ax.plot(z0, Ez0, label=r"$\Re \left( E_z\right)$", color="black")
ax.plot(
    zlist,
    reconstructed_Ez0,
    "--",
    label="reconstructed",
    color="red",
)
ax2.plot(
    zlist, abs(reconstructed_Ez0 / Ez0 - 1), "--", label="reconstructed", color="black"
)
ax2.set_ylabel("relative error")
ax2.set_yscale("log")
ax.set_ylabel("field (V/m)")
ax.set_xlabel("z (m)")
plt.legend()

This function can also be used to study the reconstruction error as a function of the number of coefficients:

In [ ]:
ncoefs = np.arange(10, FM2.shape[2] // 2)
errs = np.array(
    [
        FM2.to_impact_solrf(n_coef=n, zmirror=True)["fmap"]["info"]["Ez_err"]
        for n in ncoefs
    ]
)

fig, ax = plt.subplots()
ax.plot(ncoefs, errs, marker=".", color="black")
ax.set_xlabel("n_coef")
ax.set_ylabel("Ez reconstruction error")
ax.set_yscale("log")

## Write GPT

In [ ]:
# FM.write_gpt('solenoid.gdf', asci2gdf_bin='$ASCI2GDF_BIN', verbose=True)
FM.write_gpt("rfgun_for_gpt.txt", verbose=True)

In [ ]:
FM.write_superfish("rfgun2.t7")

# Read Superfish

Proper Superfish T7 can also be read.

In [ ]:
FM3 = FieldMesh.from_superfish("rfgun2.t7")
FM3

In [ ]:
help(FieldMesh.from_superfish)

Note that writing the ASCII and conversions alter the data slightly

In [ ]:
FM == FM3

But the data are all close:

In [ ]:
for c in FM.components:
    close = np.allclose(FM.components[c], FM3.components[c])
    equal = np.all(FM.components[c] == FM3.components[c])
    print(c, equal, close)

## Read ANSYS

Read ANSYS E and H ASCII files:

In [ ]:
FM3D = FieldMesh.from_ansys_ascii_3d(
    efile="../data/ansys_rfgun_2856MHz_E.dat",
    hfile="../data/ansys_rfgun_2856MHz_H.dat",
    frequency=2856e6,
)


FM3D

In [ ]:
FM3D.attrs

This can then be written:

In [ ]:
FM3D.write("../data/rfgun_rectangular.h5")

The y=0 plane can be extracted to be used as cylindrically symmetric data:

In [ ]:
FM2D = FM3D.to_cylindrical()
FM2D

# Cleanup

In [ ]:
import os

for file in ("test.h5", "astra_1d.dat", "rfgun_for_gpt.txt", "rfgun2.h5", "rfgun2.t7"):
    os.remove(file)